## RAG with Strands Agents SDK

This notebook demonstrates how to build a Swagger API assistant using the Strands Agents SDK. The agent can:
- Answer questions about Swagger APIs using a knowledge base
- Generate UML flow diagrams
- Create unit testing code

This is a Strands SDK implementation of the original Bedrock Agents approach, showcasing how to achieve the same functionality with a more streamlined and flexible framework.


## Pre-req
You must run the [workshop_setup.ipynb](../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

## Install Strands SDK

In [ ]:
!pip install strands-agents

## Load the parameters

In [ ]:
print("load the data parameters....\n")
# bucket and parameter stored from Initial setup lab01
%store -r bucket
%store -r prefix
%store -r data_dir
%store -r yml_dir
%store -r uml_dir

## check all 5 values are printed and do not fail
print(bucket)
print(prefix)
print(yml_dir)
print(uml_dir)
print(data_dir)

print("\nload the vector db parameters....\n")

# vector parameters stored from Initial setup lab02
%store -r vector_host
%store -r vector_collection_arn
%store -r vector_collection_id
%store -r bedrock_kb_execution_role_arn

print(vector_host)
print(vector_collection_arn)
print(vector_collection_id)
print(bedrock_kb_execution_role_arn)

print("\nload lambda parameters....\n")

%store -r lambda_arn
%store -r lambda_function_name

print(lambda_arn)
print(lambda_function_name)

print("\nload knowledge base parameters....\n")

# Load KB ID from the original notebook if it exists
%store -r kb_id
print(f"Knowledge Base ID: {kb_id}")

## Setup

In [ ]:
import boto3
import json
import uuid
from IPython.display import Markdown, display

# Strands SDK imports
from strands import Agent, tool
from strands.models import BedrockModel

# Setup boto3 session
boto3_session = boto3.Session()
region_name = boto3_session.region_name
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

# Bedrock clients
bedrock_agent_runtime_client = boto3_session.client("bedrock-agent-runtime")
bedrock_runtime = boto3_session.client("bedrock-runtime")
lambda_client = boto3.client('lambda')

print(f"Region: {region_name}")
print(f"Account ID: {account_id}")

## Define Tools for Code Generation and UML Diagrams

We'll create tools that call the same Lambda function used in the original notebook, but through Strands SDK tools.

In [ ]:
@tool
def get_uml_diagram(yml_body: str) -> str:
    """Generate a UML flow diagram referencing the OpenAPI API specification.
    
    Args:
        yml_body: openapi standard yml file of the swagger api
    """
    try:
        payload = {
            "actionGroup": "CodeAndDiagramActionGroup",
            "function": "get_uml_diagram",
            "parameters": [
                {
                    "name": "yml_body",
                    "type": "string",
                    "value": yml_body
                }
            ],
            "messageVersion": "1.0"
        }
        
        response = lambda_client.invoke(
            FunctionName=lambda_function_name,
            Payload=json.dumps(payload)
        )
        
        result = json.loads(response['Payload'].read())
        
        if 'response' in result:
            function_response = result['response']['functionResponse']['responseBody']['TEXT']['body']
            body = json.loads(function_response) if isinstance(function_response, str) else function_response
            s3_uri = body.get('diagramUri', 'Not available')
            plantuml_code = body.get('codeBody', 'Generated')
            return f"UML diagram generated successfully!\n\n**S3 Location:** {s3_uri}\n\n**PlantUML Code:**\n```plantuml\n{plantuml_code}\n```"
        else:
            return f"UML diagram generation completed: {result}"
            
    except Exception as e:
        return f"Error generating UML diagram: {str(e)}"

@tool
def get_unit_test_code(yml_body: str, user_query: str) -> str:
    """Generate functional testing code referencing the OpenAPI API specification.
    
    Args:
        yml_body: openapi standard yml file of the swagger api
        user_query: question from user or code user wants to generate
    """
    try:
        body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "messages": [{
                "role": "user",
                "content": [{
                    "type": "text",
                    "text": f"""Generate a code snippet based on this OpenAPI YAML file and user query.

YAML_FILE:
{yml_body}

USER_QUERY:
{user_query}

Generate a code snippet that demonstrates how to use the API based on the user query. Include necessary imports, endpoint URL, HTTP method, parameters, and request body. Format using markdown code blocks."""
                }]
            }]
        })
        
        response = bedrock_runtime.invoke_model(
            modelId="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
            body=body
        )
        
        result = json.loads(response.get("body").read())
        return result["content"][0]["text"]
        
    except Exception as e:
        return f"Error generating test code: {str(e)}"

@tool
def retrieve_swagger_spec(query: str) -> str:
    """Retrieve OpenAPI/Swagger specification from the knowledge base.
    
    Args:
        query: search query to find relevant API specification
    """
    try:
        response = bedrock_agent_runtime_client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 5
                }
            }
        )
        
        results = []
        for result in response['retrievalResults']:
            content = result['content']['text']
            results.append(content)
        
        return "\n\n".join(results)
        
    except Exception as e:
        return f"Error retrieving from knowledge base: {str(e)}"

## Create Strands Agent

Now we'll create a Strands agent with the Bedrock model and our custom tools.

In [ ]:
# Create Bedrock model configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name=region_name,
    temperature=0.1,
    max_tokens=4000
)

# System prompt for the agent
system_prompt = """
You are an agent designed to support users with coding questions related to Swagger APIs. You have access to the Swagger
documentation in a Knowledge Base and you can answer questions from this document or use document as reference to generate
code or UML flow diagrams. 

When users ask for:
1. UML diagrams - use the get_uml_diagram tool with the relevant OpenAPI specification
2. Test code generation - use the get_unit_test_code tool with the specification and user query
3. General questions - use retrieve_swagger_spec to get relevant documentation

Always first retrieve the relevant Swagger specification using retrieve_swagger_spec before using other tools.
Only answer questions based on the documentation and reply with "There is no information about your 
question in the Documentation at the moment, sorry! Do you want to ask another question?" if the answer to the question 
is not available in the documentation.
"""

# Create the agent
swagger_agent = Agent(
    model=bedrock_model,
    tools=[get_uml_diagram, get_unit_test_code, retrieve_swagger_spec],
    system_prompt=system_prompt,
    name="SwaggerAPIAgent"
)

print("Strands Swagger API Agent created successfully!")

## Test the Agent with Knowledge Base Retrieval

Let's test the agent's ability to retrieve information from the knowledge base.

In [ ]:
# Test basic knowledge retrieval
query = "How do I add a new pet to petstore?"
response = swagger_agent(query)
display(Markdown(str(response)))

## Test UML Diagram Generation

Now let's test the agent's ability to generate UML diagrams.

In [ ]:
# Test UML diagram generation
query = "Can you generate UML diagram for the bookstore swagger api?"
response = swagger_agent(query)
display(Markdown(str(response)))

## Test Python Code Generation

Let's test the agent's ability to generate Python test code.

In [ ]:
# Test code generation
query = "How do I add a new pet using the petstore api? Can you generate a test code in python?"
response = swagger_agent(query)
display(Markdown(str(response)))

## Advanced Usage: Streaming Responses

Strands SDK supports streaming responses for real-time interaction.

In [ ]:
# Example of streaming response
import asyncio

async def stream_example():
    query = "Explain the petstore API endpoints and generate sample code for creating a pet"
    
    print("Streaming response:")
    print("-" * 50)
    
    async for event in swagger_agent.stream_async(query):
        if content := event.get("content_block_delta", {}).get("delta", {}).get("text"):
            print(content, end="", flush=True)
    
    print("\n" + "-" * 50)
    print("Streaming complete!")

# Run the streaming example
await stream_example()

## Interactive Chat Session

Let's create an interactive chat session to demonstrate multi-turn conversations.

In [ ]:
def interactive_chat():
    """Simple interactive chat with the Swagger API agent."""
    print("Swagger API Agent Chat (type 'quit' to exit)")
    print("=" * 50)
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        
        try:
            response = swagger_agent(user_input)
            print(f"\nAgent: {str(response)}")
        except Exception as e:
            print(f"\nError: {str(e)}")

# Uncomment the line below to start interactive chat
# interactive_chat()

In [ ]:
# Test with metrics collection
query = "What are the main endpoints in the petstore API?"
response = swagger_agent(query)

# Display response
display(Markdown(str(response)))

# Display metrics
print("\n" + "=" * 50)
print("AGENT METRICS")
print("=" * 50)
print(f"Total tokens: {response.metrics.accumulated_usage.get('totalTokens', 'N/A')}")
print(f"Input tokens: {response.metrics.accumulated_usage.get('inputTokens', 'N/A')}")
print(f"Output tokens: {response.metrics.accumulated_usage.get('outputTokens', 'N/A')}")

## Cleanup (Optional)

The Strands agent doesn't require cleanup as it doesn't create persistent AWS resources. However, you may want to clean up the original Bedrock Agent resources if no longer needed.

In [ ]:
print("Strands SDK agent session complete!")
print("No cleanup required - Strands agents are stateless and don't create persistent AWS resources.")
print("\nTo clean up the original Bedrock Agent resources, refer to the original notebook.")